In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('log_reg').getOrCreate()

In [3]:
df = spark.read.csv('/FileStore/tables/customer_churn.csv', header=True, inferSchema=True)

In [4]:
df.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: timestamp (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- Churn: integer (nullable = true)

In [5]:
df.head(2)

Out[ 5 ]: 
[Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1),
 Row(Names='Kevin Mueller', Age=41.0, Total_Purchase=11916.22, Account_Manager=0, Years=6.5, Num_Sites=11.0, Onboard_date=datetime.datetime(2013, 8, 13, 0, 38, 46), Location='6157 Frank Gardens Suite 019 Carloshaven, RI 17756', Company='Wilson PLC', Churn=1)]

In [6]:
df.columns

Out[ 6 ]: 
['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [7]:
from pyspark.ml.feature import StringIndexer, VectorAssembler

In [8]:
vector_assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites'], outputCol='features')

In [9]:
output = vector_assembler.transform(df)

In [10]:
final_df = output.select(['features', 'Churn'])

In [11]:
train_df, test_df = final_df.randomSplit([.7, .3])

In [12]:
from pyspark.ml.classification import LogisticRegression

In [13]:
lr = LogisticRegression(labelCol='Churn')

In [14]:
model = lr.fit(train_df)

In [15]:
model_sum = model.summary

In [16]:
model_sum.predictions.describe().show()

+-------+------------------+------------------+
summary| Churn| prediction|
+-------+------------------+------------------+
 count| 640| 640|
 mean| 0.16875| 0.134375|
 stddev|0.3748239023147789|0.3413215364378678|
 min| 0.0| 0.0|
 max| 1.0| 1.0|
+-------+------------------+------------------+

In [17]:
test_df.columns

Out[ 17 ]: ['features', 'Churn']

In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

results = model.evaluate(test_df)

In [19]:
results.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|Churn| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[26.0,8939.61,0.0...| 0|[6.57662337648328...|[0.99860939302049...| 0.0|
[28.0,9090.43,1.0...| 0|[1.63230597278777...|[0.83648528909741...| 0.0|
[28.0,11204.23,0....| 0|[2.14440882213699...|[0.89514514540796...| 0.0|
[29.0,9617.59,0.0...| 0|[4.66688381376880...|[0.99068604452941...| 0.0|
[29.0,12711.15,0....| 0|[5.71525123536308...|[0.99671550096751...| 0.0|
[30.0,6744.87,0.0...| 0|[3.59834972747886...|[0.97336024814343...| 0.0|
[30.0,7960.64,1.0...| 1|[3.09058540986728...|[0.95650272768424...| 0.0|
[30.0,8403.78,1.0...| 0|[5.90008478667963...|[0.99726827022831...| 0.0|
[30.0,8677.28,1.0...| 0|[4.21262386347509...|[0.98540859712499...| 0.0|
[30.0,8874.83,0.0...| 0|[3.37203464590232...|[0.96681902462943...| 0.0|
[31.0,5304.6,0.0,...| 0|[3.44828774860459...|[0.96918003660112...| 0.0|
[31.0,5387.75,0.0...| 0|[2.63117075463156...|[0.93284093287797...| 0.0|
[32.0,6367.22,1.0...| 0|[2.88383624894407...|[0.94704159639998...| 0.0|
[32.0,8575.71,0.0...| 0|[3.91647674587172...|[0.98047758985952...| 0.0|
[32.0,12547.91,0....| 0|[0.85648099385070...|[0.70192490916051...| 0.0|
[32.0,13630.93,0....| 0|[2.65842286665544...|[0.93452823544054...| 0.0|
[33.0,5738.82,0.0...| 0|[4.41672616730139...|[0.98807034016359...| 0.0|
[33.0,10709.39,1....| 0|[6.28555961012732...|[0.99814045025098...| 0.0|
[33.0,12249.96,0....| 0|[5.71332047425938...|[0.99670917414776...| 0.0|
[33.0,12638.51,1....| 0|[4.02163234819772...|[0.98239191832550...| 0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [20]:
lr_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')

In [21]:
auc = lr_eval.evaluate(results.predictions)

In [22]:
auc

Out[ 22 ]: 0.7243337702053299

In [23]:
new_cust = spark.read.csv('/FileStore/tables/new_customers.csv', header=True, inferSchema=True)

In [24]:
final_lr = lr.fit(final_df)

In [25]:
pred_new_cust = vector_assembler.transform(new_cust)

In [26]:
pred_new_cust.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: timestamp (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- features: vector (nullable = true)

In [27]:
pred = final_lr.transform(pred_new_cust)

In [28]:
pred.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company| features| rawPrediction| probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
 Andrew Mccall|37.0| 9935.53| 1| 7.71| 8.0|2011-08-29 18:37:54|38612 Johnny Stra...| King Ltd|[37.0,9935.53,1.0...|[2.22168705251390...|[0.90218018099700...| 0.0|
Michele Wright|23.0| 7526.94| 1| 9.28| 15.0|2013-07-22 18:19:54|21083 Nicole Junc...| Cannon-Benson|[23.0,7526.94,1.0...|[-6.2207530595046...|[0.00198380445828...| 1.0|
 Jeremy Chang|65.0| 100.0| 1| 1.0| 15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|[65.0,100.0,1.0,1...|[-3.7691621189384...|[0.02255110110417...| 1.0|
Megan Ferguson|32.0| 6487.5| 0| 9.4| 14.0|2016-10-28 05:32:13|922 Wright Branch...| Sexton-Golden|[32.0,6487.5,0.0,...|[-5.0956222016539...|[0.00608622642083...| 1.0|
 Taylor Young|32.0| 13147.71| 1| 10.0| 8.0|2012-03-20 00:36:46|Unit 0789 Box 073...| Wood LLC|[32.0,13147.71,1....|[1.10475867224048...|[0.75115067517455...| 0.0|
 Jessica Drake|22.0| 8445.26| 1| 3.46| 14.0|2011-02-04 19:29:27|1148 Tina Straven...| Parks-Robbins|[22.0,8445.26,1.0...|[-1.6896019277075...|[0.15582819767621...| 1.0|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+

In [29]:
pred.select(['Company', 'prediction']).show()

+----------------+----------+
 Company|prediction|
+----------------+----------+
 King Ltd| 0.0|
 Cannon-Benson| 1.0|
Barron-Robertson| 1.0|
 Sexton-Golden| 1.0|
 Wood LLC| 0.0|
 Parks-Robbins| 1.0|
+----------------+----------+